net id `net1` and MLModelId `model1` do not match. Is that ony in my LV example? Check Sebastian's test cases and the format.
These should match because ude_problem.hybridization is a dict while the nets are a list.

In [57]:
from pathlib import Path
import petab_sciml
import nnUDE
import yaml
import pandas as pd

from nnUDE.neural_network import (
    Node, 
    Layer,
    NeuralNetwork,
    input_layer_from_species_ids
) 

In [23]:
fp_problem_yaml = (
    Path(".").resolve().parents[4] /
    "test_cases" /
    "published" / 
    "lv_001" / 
    "problem_ude.yaml"
)
fp_problem_yaml

PosixPath('/home/maren/petab_sciml/test_cases/published/lv_001/problem_ude.yaml')

In [24]:
problem_yaml['extensions']['petab_sciml']['net_files'][0][:-5]

'net1'

In [58]:
# import into petab_sciml

# import ude problem
with open(fp_problem_yaml) as stream:
    problem_yaml = yaml.safe_load(stream)

# get mapping table
fp_map_yaml = (
    fp_problem_yaml.parent / 
    problem_yaml['problems'][0]['mapping_tables']
)
mapping_table = pd.read_csv(fp_map_yaml, sep="\t")
inputs = mapping_table[
    mapping_table["ioId"].str.contains("input")
]["ioValue"].to_list()
outputs = mapping_table[
    mapping_table["ioId"].str.contains("output")
]["ioValue"].to_list()

# get NN specification
fn_net = problem_yaml['extensions']['petab_sciml']['net_files'][0]
fp_net_yaml = fp_problem_yaml.parent / fn_net
with open(fp_net_yaml) as stream:
    net1_yaml = yaml.safe_load(stream)

net_id = fn_net[:-5]
nn_model = net1_yaml['models'][0]

# ensure it's a FFNN
for layer in nn_model["layers"]:
    assert layer['layer_type'] == 'Linear', \
        f"Non-linear layer: {layer}"

ff_architecture = nn_model["forward"]
# first layer item should be a placeholder
assert ff_architecture[0]['op'] == 'placeholder', \
        f"First op is not 'placeholder': {nn_model['forward'][0]}"

# Is there an input transformation?
# corresp. to a function in second item
if ff_architecture[1]['op'] == "call_method":
    act = ff_architecture[1]['target']
    i = 2
elif ff_architecture[1]['op'] == "call_module":
    act = None
    i = 1
else:
    raise KeyError(f"Not a valid op: {ff_architecture[1]['op']}")

# get nUDE input layer
input_layer = input_layer_from_species_ids(
        neural_network_id=net1_yaml['models'][0]['mlmodel_id'],
        species_ids=inputs,
        # to do: allow input transformation = activation function
        # act
    )

# iterate over items in forward
# ensure at most one activation function between layers
constructed_layers = [input_layer]

while i < len(ff_architecture):
    # get layer id
    layer_id = ff_architecture[i]['name']
    layer_index_num = len(constructed_layers)
    nodes = []
    layer_info = [d for d in nn_model["layers"] if d['layer_id'] == layer_id][0]['args']
    # check for activation function
    if ff_architecture[i+1]['op'] == "call_method":
        layer_info['activation'] = ff_architecture[1]['target']
        i += 2
    elif ff_architecture[i+1]['op'] == "call_module":
        layer_info['activation'] = None
        i += 1
    elif ff_architecture[i+1]['op'] == "output":
        layer_info['activation'] = None
        i += 2
    else:
        raise KeyError(f"Not a valid op: {ff_architecture[i+1]['op']}")
    # To Do: call the NUDE layer construction function
    for node_index in range(layer_info['out_features']):
        node = Node(
            id_=f"{net_id}__layer_{layer_index_num}__node__{node_index}",
            index=node_index,
            input_nodes=constructed_layers[-1].nodes,
            activation_function=layer_info['activation'],
        )
        nodes.append(node)

    layer = Layer(
        id_=f"{net_id}__layer_{layer_id}",
        index=layer_index_num,
        nodes=nodes,
    )
    constructed_layers.append(layer)

# if i == len(ff_architecture):
#     if ff_architecture[i] == "output":
#         pass
#     else:
#         raise ValueError(f"Unexpected last item: {ff_architecture[i]}")

In [59]:
neural_network = NeuralNetwork(
    id_=net_id,
    input_layer=constructed_layers[0],
    layers=constructed_layers[1:],
    output_species_ids=outputs,
    regularization=None,
)

In [60]:
neural_network

NeuralNetwork(id_='net1', input_layer=Layer(id_='neural_network_net1__layer_input', index=None, nodes=[Node(id_='prey', index=0, input_nodes=None, activation_function=None, formula='prey'), Node(id_='predator', index=1, input_nodes=None, activation_function=None, formula='predator')]), output_species_ids=['prey_predation', 'predator_predation'], layers=[Layer(id_='net1__layer_layer1', index=1, nodes=[Node(id_='net1__layer_1__node__0', index=0, input_nodes=[Node(id_='prey', index=0, input_nodes=None, activation_function=None, formula='prey'), Node(id_='predator', index=1, input_nodes=None, activation_function=None, formula='predator')], activation_function='layer1', formula=None), Node(id_='net1__layer_1__node__1', index=1, input_nodes=[Node(id_='prey', index=0, input_nodes=None, activation_function=None, formula='prey'), Node(id_='predator', index=1, input_nodes=None, activation_function=None, formula='predator')], activation_function='layer1', formula=None)]), Layer(id_='net1__layer_l